In [11]:
import networkx as nx
import csv
from pyvis.network import Network
from IPython.display import IFrame

# Create NetworkX graph
G = nx.Graph()

with open('jobs.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        company = row["company"].strip()
        job = row["job_title"].strip()
        domain = row["domain"].strip()
        
        G.add_edge(company, job, relation="HAS_JOB")
        G.add_edge(job, domain, relation="IN_DOMAIN")
        for skill in row["domain_specific_skills"].split(","):
            skill_name = skill.strip()
            if skill_name:
                G.add_edge(job, skill_name, relation="REQUIRES_SKILL")

# Create PyVis network with physics
net = Network(notebook=True, height='750px', width='100%', cdn_resources='remote', bgcolor='#ffffff', font_color='#333333')

# Add color-coded nodes
for node in G.nodes:
    if node in [row["company"] for row in csv.DictReader(open('structured_jobs_data.csv', 'r', encoding='utf-8'))]:
        net.add_node(node, label=node, color="#1f77b4", shape="ellipse", size=30)  # Company: Blue
    elif any(node == row["job_title"] for row in csv.DictReader(open('structured_jobs_data.csv', 'r', encoding='utf-8'))):
        net.add_node(node, label=node, color="#2ca02c", shape="box", size=25)  # Job: Green
    elif any(node == row["domain"] for row in csv.DictReader(open('structured_jobs_data.csv', 'r', encoding='utf-8'))):
        net.add_node(node, label=node, color="#ff7f0e", shape="diamond", size=20)  # Domain: Orange
    else:
        net.add_node(node, label=node, color="#9467bd", shape="dot", size=15)  # Skill: Purple

# Add edges
for u, v, data in G.edges(data=True):
    net.add_edge(u, v, label=data['relation'], color="#888888")

# Improve layout - avoid overlap
net.repulsion(node_distance=200, central_gravity=0.3, spring_length=150, spring_strength=0.05)

# Save graph and display in notebook
net.save_graph("graph.html")
IFrame("temp.html", width="100%", height="750")
